In [1]:
%matplotlib inline
import sys, os, time

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import subprocess

## All of the results we have

In [2]:
splits = [
    "de_1m_2013",
    "ny_1m_2013",
    "md_1m_2013",
    "pa_1m_2013",
    "va_1m_2014",
    "wv_1m_2014"
]

In [3]:
output_dirs = [
    "../results/results_epochs_20_1",
    "../results/results_epochs_20_2",
    "../results/results_epochs_20_3",
    "../results/results_epochs_20_4"
]

## Collect all of the results using the `eval_landcover_results.sh` script

In [4]:
results_all_acc = np.zeros((len(output_dirs), len(splits), len(splits)), dtype=float)
results_all_jaccard = np.zeros((len(output_dirs), len(splits), len(splits)), dtype=float)
results_urban_acc = np.zeros((len(output_dirs), len(splits), len(splits)), dtype=float)
results_urban_jaccard = np.zeros((len(output_dirs), len(splits), len(splits)), dtype=float)

for i,output_dir in enumerate(output_dirs):
    for j,train_split in enumerate(splits):
        for k,test_split in enumerate(splits):
            command = "../eval_landcover_results.sh {}/train-output_{}/test-output_{}/log_acc_{}.txt".format(
                output_dir, train_split, test_split, test_split
            )
            text = subprocess.check_output(command.split(" "))
            lines = text.decode('utf-8').split("\n")
            all_acc, all_jaccard = list(map(float,lines[0].split(",")))
            urban_acc, urban_jaccard = list(map(float,lines[1].split(",")))
            
            results_all_acc[i,j,k] = all_acc
            results_all_jaccard[i,j,k] = all_jaccard
            results_urban_acc[i,j,k] = urban_acc
            results_urban_jaccard[i,j,k] = urban_jaccard

In [5]:
results_all_acc[-1]

array([[0.947527, 0.878812, 0.89515 , 0.896433, 0.732308, 0.31613 ],
       [0.921038, 0.926687, 0.916247, 0.908737, 0.767732, 0.462641],
       [0.927915, 0.82501 , 0.924933, 0.852362, 0.737278, 0.222232],
       [0.881993, 0.883874, 0.872149, 0.913824, 0.803648, 0.729183],
       [0.847944, 0.834645, 0.808804, 0.809283, 0.883192, 0.887725],
       [0.756144, 0.852607, 0.811233, 0.880774, 0.852514, 0.947053]])

In [6]:
df_all_acc_means = pd.DataFrame(
    results_all_acc.mean(axis=0),
    columns=["test-" + split for split in splits],
    index=["train-" + split for split in splits]
)

In [7]:
df_all_acc_stds = pd.DataFrame(
    results_all_acc.std(axis=0),
    columns=["test-" + split for split in splits],
    index=["train-" + split for split in splits]
)

In [28]:
results_acc_string = np.zeros_like(df_all_acc_means.values, dtype=object)
for i,train_split in enumerate(splits):
    for j,test_split in enumerate(splits):
        results_acc_string[i,j] = "%0.2f (%0.2f)" % (df_all_acc_means.values[i,j]*100, df_all_acc_stds.values[i,j]*100)

In [31]:
df_all_acc_string = pd.DataFrame(
    results_acc_string,
    columns=["test-" + split for split in splits],
    index=["train-" + split for split in splits]
)

In [40]:
df_all_acc_string

,test-de_1m_2013,test-ny_1m_2013,test-md_1m_2013,test-pa_1m_2013,test-va_1m_2014,test-wv_1m_2014
train-de_1m_2013,94.49 (0.17),86.38 (2.68),89.02 (0.86),88.92 (1.18),74.96 (1.22),44.09 (9.17)
train-ny_1m_2013,89.00 (3.42),89.56 (4.19),87.20 (3.65),87.94 (2.77),74.13 (3.00),35.38 (9.27)
train-md_1m_2013,88.60 (7.08),79.86 (13.81),89.99 (4.89),78.43 (17.00),69.49 (6.31),30.91 (12.22)
train-pa_1m_2013,77.12 (18.76),80.35 (12.76),78.59 (16.10),85.18 (11.01),73.74 (10.96),62.06 (13.21)
train-va_1m_2014,87.15 (1.53),86.88 (2.28),85.52 (2.75),85.72 (3.14),85.74 (5.51),78.35 (13.91)
train-wv_1m_2014,81.55 (3.78),86.51 (0.75),78.49 (4.27),87.34 (0.58),83.75 (1.75),94.22 (0.71)
